###  Introduction to Reinforcement Learning (RL) and Markov Decision Process (MDP)

## Objectives:

In this notebook, your objectives are to create environments using OpenAI-gym
and then write and evaluate simple policies(no RL training) on the created environments

The environments you will work with are "Cart Pole" and "Frozen Lake"

### Definitions

Reinforcement Learning (RL) is a branch of machine learning that focuses on learning optimal actions to take in a given environment to maximize a cumulative reward or return.

Markov Decision Process (MDP) is a mathematical framework for modeling decision-making problems in RL. It consists of states, actions, transition probabilities, and rewards.


### OpenAI Gym

[OpenAI Gym](https://www.gymlibrary.dev/) is a toolkit for developing and comparing reinforcement learning (RL) algorithms. It consists of a growing suite of environments (from simulated robots to Atari games), and a site for comparing and reproducing results. OpenAI Gym provides a diverse suite of environments that range from easy to difficult and involve many different kinds of data.

Creating and Interacting with gym environments is very simple.

```
import gym
env = gym.make("CartPole-v1")
observation, info = env.reset(seed=42)

for _ in range(1000):
    action = env.action_space.sample()
    observation, reward, done, truncated, info = env.step(action)

    if terminated or truncated:
        observation, info = env.reset()
env.close()
```

Following are the definitions of some common terminologies used.

**Reset:** Resets the environment to an initial state and returns the initial observation. <br>
**Step:** Run one timestep of the environment's dynamics.<br>
**Observation:** The observed state of the environment.<br>
**Action:** An action provided by the agent.<br>
**Reward:** The amount of reward returned as a result of taking the action.<br>
**Terminated:** Whether a terminal state (as defined under the MDP of the task) is reached.<br>
**Truncated:** Whether a truncation condition outside the scope of the MDP is satisfied. Typically a timelimit, but could also be used to indicate agent physically going out of bounds.<br>
**Info:** This contains auxiliary diagnostic information (helpful for debugging, learning, and logging).<br>
**Action Space:** This attribute gives the format of valid actions. It is of datatype Space provided by Gym. For example, if the action space is of type Discrete and gives the value Discrete(2), this means there are two valid discrete actions: 0 & 1.<br>
**Observation:** This attribute gives the format of valid observations. It is of datatype Space provided by Gym. For example, if the observation space is of type Box and the shape of the object is (4,), this denotes a valid observation will be an array of 4 numbers.<br>

Note: Previously, `terminated` and `truncated` used to be merged under one variable `done`. <br>


We will use OpenAI Gym for Frozen Lake (2D) and Cart Pole (1D) environments.

### Note:

OpenAI has launched the Gymnasium library. Gymnasium is the successor to OpenAI Gym, designed to maintain and extend the original framework for reinforcement learning environments. It offers improved support, more active development, and enhanced features while maintaining compatibility with existing Gym-based code. The interface for gymnasium and gym is almost same.

Gym itself still exists and is used, and we will stick to it in this notebook as well

# Cart Pole

|   |   |
|---|---|
| Action Space | Discrete(2) |
| Observation Shape | (4,) |
| Observation High | [4.8   inf 0.42  inf] |
| Observation Low | [-4.8   -inf -0.42  -inf] |
| Import | `gym.make("CartPole-v1")` |


### Description

This environment corresponds to the version of the cart-pole problem described by Barto, Sutton, and Anderson in
["Neuronlike Adaptive Elements That Can Solve Difficult Learning Control Problem"](https://ieeexplore.ieee.org/document/6313077).
A pole is attached by an un-actuated joint to a cart, which moves along a frictionless track.
The pendulum is placed upright on the cart and the goal is to balance the pole by applying forces
 in the left and right direction on the cart.


### Action Space

The action is a `ndarray` with shape `(1,)` which can take values `{0, 1}` indicating the direction
 of the fixed force the cart is pushed with.

| Num | Action                 |
|-----|------------------------|
| 0   | Push cart to the left  |
| 1   | Push cart to the right |


**Note**: The velocity that is reduced or increased by the applied force is not fixed and it depends on the angle
 the pole is pointing. The center of gravity of the pole varies the amount of energy needed to move the cart underneath it

### Observation Space

The observation is a `ndarray` with shape `(4,)` with the values corresponding to the following positions and velocities:

| Num | Observation           | Min                 | Max               |
|-----|-----------------------|---------------------|-------------------|
| 0   | Cart Position         | -4.8                | 4.8               |
| 1   | Cart Velocity         | -Inf                | Inf               |
| 2   | Pole Angle            | ~ -0.418 rad (-24°) | ~ 0.418 rad (24°) |
| 3   | Pole Angular Velocity | -Inf                | Inf               |

**Note:** While the ranges above denote the possible values for observation space of each element,
    it is not reflective of the allowed values of the state space in an unterminated episode. Particularly:
-  The cart x-position (index 0) can be take values between `(-4.8, 4.8)`, but the episode terminates
   if the cart leaves the `(-2.4, 2.4)` range.
-  The pole angle can be observed between  `(-.418, .418)` radians (or **±24°**), but the episode terminates
   if the pole angle is not in the range `(-.2095, .2095)` (or **±12°**)

### Rewards

Since the goal is to keep the pole upright for as long as possible, a reward of `+1` for every step taken,
including the termination step, is allotted. The threshold for rewards is 475 for v1.

### Starting State

All observations are assigned a uniformly random value in `(-0.05, 0.05)`

### Episode End

The episode ends if any one of the following occurs:

1. Termination: Pole Angle is greater than ±12°
2. Termination: Cart Position is greater than ±2.4 (center of the cart reaches the edge of the display)
3. Truncation: Episode length is greater than 500 (200 for v0)

Docs/source: https://www.gymlibrary.dev/environments/classic_control/cart_pole/

# Policies
A policy is a mapping from states to actions. It determines the action to take in each state.

In the following code cells, you will define and evaluate different policies for the 1D and 2D environments.


In [17]:
import gym
from tqdm import tqdm

import numpy as np

# For visualization
from gym.wrappers.monitoring import video_recorder
from IPython.display import HTML
from IPython import display
import glob
import base64, io, os

os.environ['SDL_VIDEODRIVER']='dummy'

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
# Function to evaluate a policy in an environment
def evaluate_policy(env, policy, num_episodes=1000):
    rewards = []
    for _ in tqdm(range(num_episodes)):
        state = env.reset()
        episode_reward = 0
        done = False
        while not done:
            action = policy(state)
            next_state, reward, done, info = env.step(action)
            episode_reward += reward
            state = next_state
        rewards.append(episode_reward)
    return sum(rewards) / num_episodes

## 1D

Create the CartPole Environment

Maybe try looking at it's observation space and action space

In [ ]:
env_1d = None

### Define CartPole Policies

Implement all the functions below to create the specify policy.

Feel free to define any extra policies you can think of.

In [15]:
# Write an angle policy: which decides the action based on the angle of the pole

def angle_policy(observation):
    # The observation here is gym's observation. Look at the action space or review the docs to
    # see how it works.
    pass

# Define the "Always Left" policy
def always_left_policy(observation):
    pass

# Define the "Always Right" policy
def always_right_policy(observation):
    pass

# Define a random policy
def random_policy(observation):
    pass

### Cartpole policy evaluations

evaluate the policies you've made above

### Visualization

You are provided with the helper code to visualize the policy on environments.
Feel free to write your own visualization code if you want

Visualize atleast one policy - the best one, for each environment and clearly label which policy you are visualizing

You can visualize more than one policies if you want but make sure you include the best policy

In [27]:
# Helper functions for visualization of policies on the environment.
# Don't modify the code in this cell

os.makedirs("video", exist_ok=True)

def show_video(env_name):
    mp4list = glob.glob('video/*.mp4')
    if len(mp4list) > 0:
        mp4 = 'video/{}.mp4'.format(env_name)
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display.display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

def show_video_of_model(env_name, env, policy, max_steps=10000, verbose=0):
    vid = video_recorder.VideoRecorder(env, path="video/{}.mp4".format(env_name))
    state = env.reset()
    done = False
    for t in tqdm(range(max_steps)):
        vid.capture_frame()
        action = policy(state)
        next_state, reward, done, info = env.step(action)
        if verbose:
            print(f"state: {state}, action: {action}",next_state, reward, done)
        state = next_state
        if done:
            break
    vid.close()
    env.close()

Visualize the policies here.
Look at the example to see how to use the provided helper code.

In [ ]:
env_name = "CartPole-v1"

show_video_of_model(env_name, env_1d, angle_policy)
show_video(env_name)

## 2D - Frozen Lake

Frozen lake is a toy text environment involves crossing a frozen lake from start to goal without falling into any holes by walking over the frozen lake. <br>

We can also set the lake to be slippery so that the agent does not always move in the intended direction. \but here, we will only look at the non-slippery case. But you are welcome to try the slippery one.<br>

You can read more about the environment [here](https://gymnasium.farama.org/environments/toy_text/frozen_lake/).

![Frozen Lake](https://gymnasium.farama.org/_images/frozen_lake.gif)


|   |   |
|---|---|
| Action Space | Discrete(4) |
| Observation Space | Discrete(16) |
| Import | `gym.make("FrozenLake-v1")` |


Frozen lake involves crossing a frozen lake from Start(S) to Goal(G) without falling into any Holes(H)
by walking over the Frozen(F) lake.
The agent may not always move in the intended direction due to the slippery nature of the frozen lake.


### Action Space
The agent takes a 1-element vector for actions.
The action space is `(dir)`, where `dir` decides direction to move in which can be:

- 0: LEFT
- 1: DOWN
- 2: RIGHT
- 3: UP

### Observation Space
The observation is a value representing the agent's current position as
current_row * nrows + current_col (where both the row and col start at 0).
For example, the goal position in the 4x4 map can be calculated as follows: 3 * 4 + 3 = 15.
The number of possible observations is dependent on the size of the map.
For example, the 4x4 map has 16 possible observations.

### Rewards

Reward schedule:
- Reach goal(G): +1
- Reach hole(H): 0
- Reach frozen(F): 0

### Arguments

```
gym.make('FrozenLake-v1', desc=None, map_name="4x4", is_slippery=True)
```

`desc`: Used to specify custom map for frozen lake. For example,

    desc=["SFFF", "FHFH", "FFFH", "HFFG"].

    A random generated map can be specified by calling the function `generate_random_map`. For example,

    ```
    from gym.envs.toy_text.frozen_lake import generate_random_map

    gym.make('FrozenLake-v1', desc=generate_random_map(size=8))
    ```

`map_name`: ID to use any of the preloaded maps.

    "4x4":[
        "SFFF",
        "FHFH",
        "FFFH",
        "HFFG"
        ]

    "8x8": [
        "SFFFFFFF",
        "FFFFFFFF",
        "FFFHFFFF",
        "FFFFFHFF",
        "FFFHFFFF",
        "FHHFFFHF",
        "FHFFHFHF",
        "FFFHFFFG",
    ]

`is_slippery`: True/False. If True will move in intended direction with
probability of 1/3 else will move in either perpendicular direction with
equal probability of 1/3 in both directions.

    For example, if action is left and is_slippery is True, then:
    - P(move left)=1/3
    - P(move up)=1/3
    - P(move down)=1/3


Create the FrozenLake Environment

In [ ]:
env_2d = None

Implement the following policy functions.

Feel free to implement any extra policy if you want.

In [33]:
# Define the "Always Left" policy
def always_left_policy(observation):
    pass

# Define the "Always Right" policy
def always_right_policy(observation):
    pass

# Define a random policy
def random_policy(observation):
    pass

# Define a custom algorithm(one you think will work) which uses
# the positions of agent and the goal to reach the goal
def position_based_policy(observation):
    pass

### FrozenLake policy evaluations

evaluate the policies you've made above

### Visualization

Visualize the custom position policy you made, as well as any other you want

clearly label the visualiztions with the policies

In [ ]:
# Random policy

env_name = "lake-random"

show_video_of_model(env_name, env_2d, random_policy)
show_video(env_name)